In [54]:
# !pip3.6 install paramiko

In [55]:
import configparser
import paramiko

In [56]:
# file_path = '/opt/webro/api-config.properties'
# file_path = '/opt/batchMyTimes/mongoConfig.properties'
# file_path = '/opt/webro/api-config.properties'
file_path = '/opt/mytimes-props/prop/mytimes.properties'
# file_path = '/opt/database.properties'
# file_path = '/opt/batchMyTimes/appConfig.properties'
# file_path = '/opt/kafka/config/producer.properties'

# server_1 = '172.29.16.100'
server_1 = '172.29.16.120'
# server_1 = '172.24.82.94'
# server_1 = '192.168.43.240'
# server_1 = '172.24.80.140'
# server_1 = '192.168.42.96'

# server_2 = '172.24.82.94'
# server_2 = '172.24.80.141'
# server_2 = '192.168.43.239'
# server_2 = '172.24.81.47'
server_2 = '192.168.43.239'
# server_2 = None

server_list = None
# server_list = '172.24.80.28,172.24.80.29,172.29.64.32,172.29.64.34,172.29.64.41'.split(',')
# server_list = ['192.168.43.239', '192.168.43.196', '192.168.43.195', '192.168.43.240', '192.168.43.197']

# server_list = ['192.168.43.195','192.168.43.196','192.168.43.197','192.168.36.127','172.29.16.155','172.29.16.156','172.29.16.157']
# server_list = ['192.168.43.239','192.168.43.195']

# mytpvt servers
# server_list = '172.24.80.140,172.24.80.141,172.24.80.142,172.29.64.131,172.29.64.133'.split(',')

In [57]:
DUMMY_SECTION_NAME = 'dummy_section'

def get_generic_config(prop_content, section_name = DUMMY_SECTION_NAME):
    """
    prop_content - file content of the property file
    """
    config_string = '[' + section_name + ']\n' + prop_content
    config = configparser.RawConfigParser()
    config.optionxform = str
    config.read_string(config_string)
    return config

def get_dict_config(config, section_name = DUMMY_SECTION_NAME):
    return dict(config._sections[section_name])

In [58]:
def get_remote_file_content(server_ip, file_path, username, password):
    with paramiko.SSHClient() as ssh_client:
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(hostname=server_ip,username=username ,password=password)
        with ssh_client.open_sftp() as sftp_client:
            with sftp_client.open(file_path) as remote_file:
                content = remote_file.read()
                content = content.decode('utf-8')
    return content

In [59]:
ssh_config_file_loc = '/Users/yash.dalmia/jupyter_notebooks/config/config.ini'

config_ssh = configparser.ConfigParser()
config_ssh.read(ssh_config_file_loc)

['/Users/yash.dalmia/jupyter_notebooks/config/config.ini']

In [60]:
username=config_ssh['LOGIN']['username']
password=config_ssh['LOGIN']['password']

def get_config_dict_server(server_ip, file_path = file_path, username = username, password = password):
    if(server_ip):
        file_content = get_remote_file_content(server_ip, file_path, username, password)
    else:
        file_content = open(file_path).read()
    config_file = get_generic_config(file_content)
    dict_config = get_dict_config(config_file)
    return dict_config

In [61]:
# print(username, password)

In [62]:
# list of servers property check
# print the servers with prop diff with the reference server which is taken as server_list[0]


In [63]:
def get_dict_props_server_list(server_list):
    """
    returns a dictionary of server and server properties set
    """
    dict_set_props = dict()
    for server in server_list:
        set_props = set(get_config_dict_server(server).items())
        dict_set_props[server] = set_props
    return dict_set_props

In [64]:
def find_server_pros_mismatch(server_list, ref_server_index = 0):
    """
    prints the server with property mismatch with the reference server property
    """
    ref_server = server_list[ref_server_index]
    print('reference server : {}'.format(ref_server))
    dict_set_props = get_dict_props_server_list(server_list)
    ref_props = dict_set_props[ref_server]
    for server, props in dict_set_props.items():
        if props!= ref_props:
            print('server {} mismatch'.format(server))

In [65]:
dict_config_1 = get_config_dict_server(server_1)
# dict_config_1 = get_config_dict_server(server_ip = None, file_path = "/Users/yash.dalmia/mytimes.properties_239_backup")
dict_config_2 = get_config_dict_server(server_2)
# dict_config_2 = get_config_dict_server(server_ip = None, file_path = "/Users/yash.dalmia/mytimes.properties_196_backup")
set_1 = set(dict_config_1.items())
set_2 = set(dict_config_2.items())

In [66]:
set_1

{('Ip.filter', '192.169.31.57,223.165.28.230'),
 ('accesskey', '76a73a38ad82ef54ea902e7061cb5269'),
 ('activate.user.profile.baseurl',
  'https://172.29.61.45/mytimes/activateProfile'),
 ('authorDetail.isSolr', 'true'),
 ('base.rewards.point.url',
  'http://rewards.indiatimes.com/bp/api/urs/mubhtry?'),
 ('base.rewards.point.url.secure',
  'https://rewards.indiatimes.com/bp/api/urs/mubhtry?'),
 ('base.user.rewards.url', 'http://rewards.indiatimes.com/bp/api/urs/ups?'),
 ('base.user.rewards.url.secure',
  'https://rewards.indiatimes.com/bp/api/urs/ups?'),
 ('blocked.controllers',
  '/getFeed/User,/myActivity,/userActivity,/activity/myfriends,/author/comments,/author/entity,/notification,/topic/detail,/topic/search,/topics/all,/qna/category/all,/qna/question/all,/user/search,/user/authorList,/searchUser,/activity/comments,/activity/likes,/activity/share,/qna/questionsDetail,/qna/questionFollower,/qna/catDetail,/friendList,/followee/count,/facepiles/activity,/topicFollower,/activityFollowe

In [67]:
print('server 1: {}\nserver 2: {}'.format(server_1, server_2))

server 1: 172.29.16.120
server 2: 192.168.43.239


In [68]:
key_set_diff_1_2 = {x[0] for x in (set_1-set_2)}
key_set_diff_2_1 = {x[0] for x in (set_2-set_1)}

In [69]:
##printing all the differnces between the property file of the two servers in a beautified format 
print('{:50}   {:^30} : {:^30}\n'.format('', server_1, server_2 if server_2 else 'local'))

#iterates over those keys which are present on both the servers
for common_key in (key_set_diff_1_2 & key_set_diff_2_1):
    print('{:50} : {:^30} : {:^30}'.format(common_key, dict_config_1[common_key], dict_config_2[common_key]))

#iterates over only those keys that are present on server 1 and not on server 2
for key in (key_set_diff_1_2 - key_set_diff_2_1):
    print('{:50} : {:^30} : {:^30}'.format(key, dict_config_1[key], ''))

#iterates over only those keys that are present on server 2 and not on server 1
for key in (key_set_diff_2_1 - key_set_diff_1_2):
    print('{:50} : {:^30} : {:^30}'.format(key, '', dict_config_2[key]))

                                                             172.29.16.120          :         192.168.43.239        

kafka.consumer.elasticsearch.topic.concurrency     :               30               :               70              
mytimes.list.all.servers                           :  172.29.16.120,172.29.243.24   : 192.168.43.195,192.168.43.196,192.168.43.197,192.168.36.127,172.29.16.155,172.29.16.156,172.29.16.157,192.168.43.239,192.168.43.240,172.24.82.94,172.24.81.47
update.user.profile.url                            : https://172.29.61.45/mytimes/profile/update : https://npcomments.indiatimes.com/mytimes/profile/update
credit.timespoint.uri                              : https://tpapi.timespoints.com/tp/v1/apush : http://tpapi.timespoints.com/tp/v1/apush
enable.remove.activity.in.sych.for.batch.archiving.process :              true              :             false             
mytimes.rabbit.password                            :             guest              :         Mytimes@

In [70]:
#printing the servers with property diff with the server_list[index]
if server_list:
    find_server_pros_mismatch(server_list, 1)